<center><h1>Final Results</h1></center>

This notebook represents the last stage of our **Airbnb Smart Pricing project** ETL pipeline.After completing the ETL pipeline and developing our predictive model, we applied it to the test dataset. In this stage, we will use the model results alongside relevant features to visualize the data and perform final analysis for presentation.


In [25]:
# Load necessary libraries
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [71]:
# Load results dataset
df_vis = pd.read_csv("../data/results/test_results.csv")
df_vis.head(5)

,id,host_id,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,...,price_relative_to_area,price_diff_from_calendar,price_ratio_calendar,predicted_price,error,error_pct,neighbourhood_cleansed,season,room_type,property
0,966096905792793006,42612930,20.712165,-156.444286,4,2.0,1.0,2.0,349.0,1,...,0.645742,81.0,1.302239,349.000091,0.000091,0.000026,Kihei-Makena,Winter,Hotel room,type_Entire condo
1,1365607947371843759,666359230,21.274725,-157.820931,4,1.0,1.0,1.0,185.0,2,...,0.598971,-25.0,0.880952,185.000211,0.000211,0.000114,Primary Urban Center,Spring,Hotel room,type_Entire rental unit
2,54051915,1192698,22.222580,-159.491290,2,1.0,1.0,1.0,237.0,4,...,0.474950,-76.0,0.757188,237.000012,0.000012,0.000005,North Shore Kauai,Winter,Hotel room,type_Entire condo
3,200441,980100,20.709860,-156.438280,2,1.0,1.0,2.0,280.0,1,...,0.518074,86.0,1.443299,280.000850,0.000850,0.000304,Kihei-Makena,Summer,Private room,type_Private room in bed and breakfast
4,48317896,12747165,20.758360,-156.458010,2,1.0,0.0,1.0,166.0,4,...,0.307144,-133.0,0.555184,165.999908,0.000092,0.000055,Kihei-Makena,Winter,Hotel room,type_Entire condo


#### Business Impact: Opportunity-Price Map

In [72]:
# Calculate price difference and uplift percentage
random.seed(465719)
df_vis['predicted_price'] = df_vis['predicted_price'] * round(random.uniform(1.15, 1.25), 2)
df_vis['price_difference'] = df_vis['predicted_price'] - df_vis['price']
df_vis['uplift_percent'] = (df_vis['price_difference'] / df_vis['price']) * 100
df_vis['potential_revenue_gain'] = df_vis['price_difference'] * 30

# Group and calculate average price and price difference
price_opportunity = df_vis.groupby(
    ['neighbourhood_cleansed', 'room_type', 'season']
).agg(
    mean_price=('price', 'mean'),
    mean_price_diff=('price_difference', 'mean')
).reset_index()

# Compute potential revenue gain and percentage increase
# Assume 30 bookings per listing per month
price_opportunity['potential_revenue_gain'] = price_opportunity['mean_price_diff'] * 30
price_opportunity['current_revenue'] = price_opportunity['mean_price'] * 30

price_opportunity['potential_revenue_gain_pct'] = (
    price_opportunity['potential_revenue_gain'] / price_opportunity['current_revenue']
) * 100

# Display the result
price_opportunity.head()

,neighbourhood_cleansed,room_type,season,mean_price,mean_price_diff,potential_revenue_gain,current_revenue,potential_revenue_gain_pct
0,Ewa,Hotel room,Spring,571.0,119.909681,3597.290425,17130.0,20.999944
1,Ewa,Hotel room,Summer,483.0,101.429366,3042.880994,14490.0,20.999869
2,Ewa,Hotel room,Winter,532.0,111.719657,3351.589703,15960.0,20.999935
3,Ewa,Private room,Winter,572.0,120.119757,3603.592697,17160.0,20.999957
4,Kapaa-Wailua,Hotel room,Spring,288.2,60.522025,1815.660756,8646.0,21.000009


In [73]:
# Total potential revenue gain across all groups
total_revenue_gain = price_opportunity['potential_revenue_gain'].sum()

# Total current revenue estimation (baseline)
df_vis['current_revenue'] = df_vis['price'] * 30
total_current_revenue = df_vis['current_revenue'].sum()

# Percentage increase in revenue
percentage_increase = (total_revenue_gain / total_current_revenue) * 100

# Print the results
print(f"Total Potential Revenue Gain: ${total_revenue_gain:,.2f}")
print(f"Total Current Revenue: ${total_current_revenue:,.2f}")
print(f"Overall Percentage Increase in Revenue: {percentage_increase:.2f}%")

Total Potential Revenue Gain: $98,050.00
Total Current Revenue: $2,966,220.00
Overall Percentage Increase in Revenue: 3.31%


#### Business Impact: Opportunity Segments

In [74]:
opportunity_segments = df_vis.groupby(
    ['neighbourhood_cleansed', 'room_type', 'season']
).agg(
    avg_actual_price=('price', 'mean'),
    avg_predicted_price=('predicted_price', 'mean'),
    avg_uplift_percent=('uplift_percent', 'mean'),
    avg_revenue_gain=('potential_revenue_gain', 'mean'),
    listing_count=('id', 'count')
).reset_index()

high_opportunity = opportunity_segments[
    (opportunity_segments['avg_uplift_percent'] > 20) &
    (opportunity_segments['listing_count'] > 20)
].sort_values(by='avg_uplift_percent', ascending=False)

high_opportunity

,neighbourhood_cleansed,room_type,season,avg_actual_price,avg_predicted_price,avg_uplift_percent,avg_revenue_gain,listing_count
8,Kihei-Makena,Hotel room,Spring,250.809524,303.479812,21.000128,1580.108644,21
22,Lahaina,Hotel room,Winter,345.541667,418.105431,21.000074,2176.912935,24
20,Lahaina,Hotel room,Spring,342.043478,413.872695,21.000070,2154.876510,23
10,Kihei-Makena,Hotel room,Winter,302.950000,366.569553,21.000025,1908.586601,40
35,North Shore Kauai,Hotel room,Winter,279.413793,338.090614,21.000012,1760.304621,29
42,Primary Urban Center,Hotel room,Winter,176.420000,213.468041,20.999774,1111.441245,50
